In [14]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [15]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_pytorch = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader_pytorch = torch.utils.data.DataLoader(mnist_pytorch, batch_size=1, shuffle=False)

In [16]:
# Define the CNN model
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the transforms and load the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

# Initialize the model, loss function, and optimizer
model = BasicCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
# Train the model
for epoch in range(10):  # number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 0.664
[Epoch 1, Batch 200] loss: 0.150
[Epoch 1, Batch 300] loss: 0.094
[Epoch 1, Batch 400] loss: 0.094
[Epoch 2, Batch 100] loss: 0.066
[Epoch 2, Batch 200] loss: 0.053
[Epoch 2, Batch 300] loss: 0.060
[Epoch 2, Batch 400] loss: 0.053
[Epoch 3, Batch 100] loss: 0.038
[Epoch 3, Batch 200] loss: 0.043
[Epoch 3, Batch 300] loss: 0.035
[Epoch 3, Batch 400] loss: 0.039
[Epoch 4, Batch 100] loss: 0.025
[Epoch 4, Batch 200] loss: 0.030
[Epoch 4, Batch 300] loss: 0.029
[Epoch 4, Batch 400] loss: 0.033
[Epoch 5, Batch 100] loss: 0.017
[Epoch 5, Batch 200] loss: 0.030
[Epoch 5, Batch 300] loss: 0.027
[Epoch 5, Batch 400] loss: 0.024
[Epoch 6, Batch 100] loss: 0.018
[Epoch 6, Batch 200] loss: 0.021
[Epoch 6, Batch 300] loss: 0.018
[Epoch 6, Batch 400] loss: 0.021
[Epoch 7, Batch 100] loss: 0.013
[Epoch 7, Batch 200] loss: 0.015
[Epoch 7, Batch 300] loss: 0.017
[Epoch 7, Batch 400] loss: 0.018
[Epoch 8, Batch 100] loss: 0.014
[Epoch 8, Batch 200] loss: 0.010
[Epoch 8, 

In [18]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 99.20%
